## Effect of national factors on home prices in USA

**Task -** Using publically available data for the national factors that impact supply and demand of homes in US, building a model to study the effect of these variables on home prices.

**Approach -** The following variables are chosen for the study-
- Unemployment Rate
- Per Capita GDP
- Median Household Income
- Construction Prices
- CPI 
- Interest Rates
- Number of new houses supplied
- Working Population
- Urban Population
- Percentage of population above 65
- Housing subsidies
- Number of Households

As a proxy to the home prices, S&P CASE-SHILLER Index is used. 

Most of the data is downloaded from [https://fred.stlouisfed.org/].

Data for all the variables is downloaded, preprocessed and combined to create a datset. Data for different variables had different frequencies. So, to combine the data, necessary interpolations are made.

Linear Regression is used as most of the variables have high correlation with the target variable.

In [1]:
# importing libraries
import numpy as np
import pandas as pd

In [5]:
#Reading CASE-SHILLER Index into a dataframe
df_CS = pd.read_csv("CASESHILLER.csv")


#Changing dtype of date column
df_CS["DATE"] = pd.to_datetime(df_CS["DATE"])

#Selecting data post JUNE 2001
mask = df_CS["DATE"] >= "2001-07-01"
df_CS = df_CS[mask]

#Resetting Index
df_CS.reset_index(inplace = True)
df_CS.drop(columns = ["index"], inplace = True)

# Creating "Year" and "Month" columns
df_CS["Year"] = pd.DatetimeIndex(df_CS["DATE"]).year
df_CS["Month"] = pd.DatetimeIndex(df_CS["DATE"]).month
print(df_CS.shape)
df_CS.tail()

(249, 4)


,DATE,CSUSHPISA,Year,Month
244,2023-05-01,302.566,2023,5
245,2023-06-01,304.593,2023,6
246,2023-07-01,306.767,2023,7
247,2023-08-01,309.155,2023,8
248,2023-09-01,311.175,2023,9


In [9]:
# Reading Unemployment Rate Data into a dataframe
df_unemp = pd.read_csv("UNRATE.csv")
df_unemp.drop([249,250], inplace = True)
print(df_unemp.shape)
df_unemp.tail()

(249, 2)


,DATE,UNRATE
244,2023-05-01,3.7
245,2023-06-01,3.6
246,2023-07-01,3.5
247,2023-08-01,3.8
248,2023-09-01,3.8


In [10]:
# Reading Per Capita GDP Data into a dataframe
df_pcgdp = pd.read_csv("gdp.csv", names = ["DATE", "Per_Capita_GDP"], skiprows = 1)
print(df_pcgdp.shape)
df_pcgdp.tail()

(83, 2)


,DATE,Per_Capita_GDP
78,2022-07-01,25994.639
79,2022-10-01,26408.405
80,2023-01-01,26813.601
81,2023-04-01,27063.012
82,2023-07-01,27644.463


The data is quarterly. I will impute for other months using linear interpolation after we create the final dataframe combining all the data.

In [11]:
# Interest Rate Data
df_Fed_rate = pd.read_csv("FEDFUNDS.csv").drop([249,250])
print(df_Fed_rate.shape)
df_Fed_rate.tail()

(249, 2)


,DATE,FEDFUNDS
244,2023-05-01,5.06
245,2023-06-01,5.08
246,2023-07-01,5.12
247,2023-08-01,5.33
248,2023-09-01,5.33


In [14]:
# Consumer Price Index
df_CPI = pd.read_csv("CPIAUCSL.csv", names = ["DATE", "CPI"], skiprows = 1).drop([249])
print(df_CPI.shape)
df_CPI.tail()

(249, 2)


,DATE,CPI
244,2023-05-01,303.294
245,2023-06-01,303.841
246,2023-07-01,304.348
247,2023-08-01,306.269
248,2023-09-01,307.481


In [15]:
# Population above 65

df_oldpop = pd.read_csv("POP.csv", names = ["DATE", "old_percent"], skiprows = 1)
print(df_oldpop.shape)
df_oldpop.tail()

(252, 2)


,DATE,old_percent
247,2023-08-01,335413.676
248,2023-09-01,335587.795
249,2023-10-01,335761.692
250,2023-11-01,335911.267
251,2023-12-01,336047.315


In [22]:
# Merging Per Capita GDP (Quarterly data)
df_CS["DATE"] = pd.to_datetime(df_pcgdp["DATE"])
df_CS = pd.merge(df_CS,df_pcgdp, how = "left")


In [442]:
# Merging other dataframes 
others = [df_urban, df_households, df_income, df_subsidy, df_oldpop]
for df1 in others:
    if "Year" not in df1.columns:
        df1["Year"] = pd.DatetimeIndex(df1["DATE"]).year
        df1.set_index("DATE", inplace = True)
        df = pd.merge(df, df1, how = "left", on = "Year")
    else:
        df1.set_index("DATE", inplace = True)
        df = pd.merge(df, df1, how = "left", on = "Year")
df["DATE"] = df_CS["DATE"]
df.set_index("DATE", inplace = True)
df.head()

,CSUSHPISA,Year,Month,Per_Capita_GDP,Working_Population,Houses,CPI,UNRATE,Cons_Materials,FEDFUNDS,Urban_pop,Num_Households,Income,Subsidy,old_percent
DATE,,,,,,,,,,,,,,,
2001-07-01,113.491,2001,7,46390.0,181624734.6,4.2,177.4,4.6,143.4,3.77,79.057,108209.0,61889.0,20.573,12.313929
2001-08-01,114.166,2001,8,NaN,181840987.1,4.4,177.4,4.9,143.3,3.65,79.057,108209.0,61889.0,20.573,12.313929
2001-09-01,114.811,2001,9,NaN,182058514.8,4.4,178.1,5.0,143.0,3.07,79.057,108209.0,61889.0,20.573,12.313929
2001-10-01,115.308,2001,10,46400.0,182240152.0,4.3,177.6,5.3,142.3,2.49,79.057,108209.0,61889.0,20.573,12.313929
2001-11-01,115.855,2001,11,NaN,182436924.9,4.1,177.5,5.5,142.1,2.09,79.057,108209.0,61889.0,20.573,12.313929


In [443]:
df.isna().sum()

CSUSHPISA               0
Year                    0
Month                   0
Per_Capita_GDP        168
Working_Population      0
Houses                  0
CPI                     0
UNRATE                  0
Cons_Materials          0
FEDFUNDS                0
Urban_pop               0
Num_Households         18
Income                 18
Subsidy                18
old_percent             6
dtype: int64

The "Per_Capita_GDP" column has missing values because the data was quarterly. The missing values in the other columns is due to unavailability of fresh data. We will first fill the missing values in the "Per_Capita_GDP" column using linear interpolation. We will drop the rows having missing values in the other columns. This means that we will use data from 2001 to 2020

In [444]:
# Filling missing values in the Per_Capita_GDP column using linear interpolation
df["Per_Capita_GDP"] = df["Per_Capita_GDP"].interpolate()

In [445]:
df.head()

,CSUSHPISA,Year,Month,Per_Capita_GDP,Working_Population,Houses,CPI,UNRATE,Cons_Materials,FEDFUNDS,Urban_pop,Num_Households,Income,Subsidy,old_percent
DATE,,,,,,,,,,,,,,,
2001-07-01,113.491,2001,7,46390.000000,181624734.6,4.2,177.4,4.6,143.4,3.77,79.057,108209.0,61889.0,20.573,12.313929
2001-08-01,114.166,2001,8,46393.333333,181840987.1,4.4,177.4,4.9,143.3,3.65,79.057,108209.0,61889.0,20.573,12.313929
2001-09-01,114.811,2001,9,46396.666667,182058514.8,4.4,178.1,5.0,143.0,3.07,79.057,108209.0,61889.0,20.573,12.313929
2001-10-01,115.308,2001,10,46400.000000,182240152.0,4.3,177.6,5.3,142.3,2.49,79.057,108209.0,61889.0,20.573,12.313929
2001-11-01,115.855,2001,11,46494.333333,182436924.9,4.1,177.5,5.5,142.1,2.09,79.057,108209.0,61889.0,20.573,12.313929


In [446]:
df.dropna(inplace = True)

In [447]:
df.isna().sum()

CSUSHPISA             0
Year                  0
Month                 0
Per_Capita_GDP        0
Working_Population    0
Houses                0
CPI                   0
UNRATE                0
Cons_Materials        0
FEDFUNDS              0
Urban_pop             0
Num_Households        0
Income                0
Subsidy               0
old_percent           0
dtype: int64

In [448]:
df.shape

(234, 15)

In [449]:
df.tail()

,CSUSHPISA,Year,Month,Per_Capita_GDP,Working_Population,Houses,CPI,UNRATE,Cons_Materials,FEDFUNDS,Urban_pop,Num_Households,Income,Subsidy,old_percent
DATE,,,,,,,,,,,,,,,
2020-08-01,222.507,2020,8,56133.000000,205827085.3,3.3,259.580,8.4,240.9,0.10,82.459,128451.0,67521.0,44.145,16.630926
2020-09-01,225.756,2020,9,56333.000000,206122572.1,3.4,260.190,7.9,246.9,0.09,82.459,128451.0,67521.0,44.145,16.630926
2020-10-01,229.409,2020,10,56533.000000,206068465.2,3.4,260.352,6.9,246.4,0.09,82.459,128451.0,67521.0,44.145,16.630926
2020-11-01,232.726,2020,11,56823.666667,206125599.5,4.1,260.721,6.7,244.3,0.09,82.459,128451.0,67521.0,44.145,16.630926
2020-12-01,235.776,2020,12,57114.333333,205988747.7,4.1,261.564,6.7,248.0,0.09,82.459,128451.0,67521.0,44.145,16.630926


This is our preprocessed datset. Let's save it as "prepared_dataset.csv".

In [450]:
df.to_csv("prepared_dataset.csv")

In [452]:
df = pd.read_csv("prepared_dataset.csv").set_index("DATE")
df.head()

,CSUSHPISA,Year,Month,Per_Capita_GDP,Working_Population,Houses,CPI,UNRATE,Cons_Materials,FEDFUNDS,Urban_pop,Num_Households,Income,Subsidy,old_percent
DATE,,,,,,,,,,,,,,,
2001-07-01,113.491,2001,7,46390.000000,181624734.6,4.2,177.4,4.6,143.4,3.77,79.057,108209.0,61889.0,20.573,12.313929
2001-08-01,114.166,2001,8,46393.333333,181840987.1,4.4,177.4,4.9,143.3,3.65,79.057,108209.0,61889.0,20.573,12.313929
2001-09-01,114.811,2001,9,46396.666667,182058514.8,4.4,178.1,5.0,143.0,3.07,79.057,108209.0,61889.0,20.573,12.313929
2001-10-01,115.308,2001,10,46400.000000,182240152.0,4.3,177.6,5.3,142.3,2.49,79.057,108209.0,61889.0,20.573,12.313929
2001-11-01,115.855,2001,11,46494.333333,182436924.9,4.1,177.5,5.5,142.1,2.09,79.057,108209.0,61889.0,20.573,12.313929
